# Лабораторная работа №6. Методы заполнения пропусков.
## Вариант 1.
## Булыгин Максим

Подключаем необходимые библиотеки.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn import preprocessing
from sklearn.decomposition import PCA
import seaborn as sns
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from sklearn.impute import SimpleImputer, KNNImputer
from missingpy import MissForest

Импортируем данные.

In [4]:
data = pd.read_csv('bank-additional-full.csv', sep=';')
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


## Первичная обработка данных

Смотрим общую информацию и описательные статистики датасета.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
age               41188 non-null int64
job               41188 non-null object
marital           41188 non-null object
education         41188 non-null object
default           41188 non-null object
housing           41188 non-null object
loan              41188 non-null object
contact           41188 non-null object
month             41188 non-null object
day_of_week       41188 non-null object
duration          41188 non-null int64
campaign          41188 non-null int64
pdays             41188 non-null int64
previous          41188 non-null int64
poutcome          41188 non-null object
emp.var.rate      41188 non-null float64
cons.price.idx    41188 non-null float64
cons.conf.idx     41188 non-null float64
euribor3m         41188 non-null float64
nr.employed       41188 non-null float64
y                 41188 non-null object
dtypes: float64(5), int64(5), object(11)
memory usa

In [6]:
data.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


Судя по описательным статистикам, выбросов в числовых факторах нет.

Как видим, у нас 21 фича - 10 из которых числовые, 10 категориальные и одна - отклик (y).

Проверим, есть ли пропущенные значения.

In [7]:
data.isnull().sum()

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [8]:
data[data.isna().any(axis=1)]

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y


Пустых значений (null) в датасете нет.

In [9]:
data.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

Проверим уникальные значения категориальных фич.

In [10]:
categorial = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week']

In [11]:
for feat in categorial:
    print(data[feat].unique())

['housemaid' 'services' 'admin.' 'blue-collar' 'technician' 'retired'
 'management' 'unemployed' 'self-employed' 'unknown' 'entrepreneur'
 'student']
['married' 'single' 'divorced' 'unknown']
['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course'
 'unknown' 'university.degree' 'illiterate']
['no' 'unknown' 'yes']
['no' 'yes' 'unknown']
['no' 'yes' 'unknown']
['telephone' 'cellular']
['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
['mon' 'tue' 'wed' 'thu' 'fri']


Видим, что в 7 категориальных факторах есть уникальное значение 'unknown', которое соответствует пропущенному значению. Возникает вопрос - что с ними делать: принимать за выбросы, или оставлять как есть?

In [12]:
data[(data['default'] == 'unknown') | (data['housing'] == 'unknown') | (data['marital'] == 'unknown') 
     | (data['education'] == 'unknown') | (data['job'] == 'unknown') | (data['loan'] == 'unknown')
    | (data['contact'] == 'unknown') | (data['month'] == 'unknown') | (data['day_of_week'] == 'unknown')]

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
5,45,services,married,basic.9y,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
7,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
10,41,blue-collar,married,unknown,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
15,54,retired,married,basic.9y,unknown,yes,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41118,34,technician,married,unknown,no,yes,no,cellular,nov,tue,...,2,999,2,failure,-1.1,94.767,-50.8,1.046,4963.6,no
41120,60,admin.,married,unknown,no,no,no,cellular,nov,tue,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.046,4963.6,no
41122,34,technician,married,unknown,no,no,no,cellular,nov,tue,...,3,999,0,nonexistent,-1.1,94.767,-50.8,1.046,4963.6,yes
41135,54,technician,married,unknown,no,yes,no,cellular,nov,thu,...,1,999,1,failure,-1.1,94.767,-50.8,1.041,4963.6,no


Видим, что всего наблюдений, в которых есть хотя-бы одно пропущенное значение (т.е. 'unknown') всего 10700, что составляет чуть больше четверти от исходного размера датасета, поэтому исключать все эти наблюдения, из за пропущенных значений в хотя-бы одном поле - неразумно, поскольку в таком случае мы потеряем порядка четверти всех наблюдений.

Так как в данной лабораторной работе подразумевается построение первой модели по данным, в которых нет пропусков - проведем очистку данных от пропусков.

Посмотрим отдельно по фичам, сколько в каждой категориальной фиче наблюдений с 'unknown'.

In [13]:
for feat in categorial:
    print(f"Фича {feat} имеет {len(data[data[feat] == 'unknown'])} полей unknown")

Фича job имеет 330 полей unknown
Фича marital имеет 80 полей unknown
Фича education имеет 1731 полей unknown
Фича default имеет 8597 полей unknown
Фича housing имеет 990 полей unknown
Фича loan имеет 990 полей unknown
Фича contact имеет 0 полей unknown
Фича month имеет 0 полей unknown
Фича day_of_week имеет 0 полей unknown


In [14]:
data.default.unique()

array(['no', 'unknown', 'yes'], dtype=object)

Видим, что категориальная переменная default имеет 8597 полей unknown - что составляет чуть меньше четверти от размера датасета, причем эта переменная - бинарная (yes/no), поэтому включение/невключение этой переменной в набор предикторов может существенно повлиять на полученный результат. В данной лабораторной работе, попробуем исключить переменную default из набора предикторов.

In [15]:
data = data.drop(columns=['default'])
data

,age,job,marital,education,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


Что же делать с другими категориальными фичами с пропусками?

In [16]:
data.job.unique()

array(['housemaid', 'services', 'admin.', 'blue-collar', 'technician',
       'retired', 'management', 'unemployed', 'self-employed', 'unknown',
       'entrepreneur', 'student'], dtype=object)

In [17]:
data.marital.unique()

array(['married', 'single', 'divorced', 'unknown'], dtype=object)

In [18]:
data.education.unique()

array(['basic.4y', 'high.school', 'basic.6y', 'basic.9y',
       'professional.course', 'unknown', 'university.degree',
       'illiterate'], dtype=object)

Переменные job, marital, education имеют большое (>2) количество уникальных значений, поэтому, исключать их из рассмотрения неразумно - поэтому, удалим из исходного набора данных 330 + 80 наблюдений с пропущенным значением job, marital, а также удалим 1731 пропуск в переменной education.

In [19]:
data = data[(data.job != 'unknown') & (data.marital != 'unknown') & (data.education != 'unknown')]
data

,age,job,marital,education,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [20]:
data.housing.unique()

array(['no', 'yes', 'unknown'], dtype=object)

In [21]:
data.loan.unique()

array(['no', 'yes', 'unknown'], dtype=object)

In [22]:
print(len(data[(data.loan == 'unknown') | (data.housing == 'unknown')]))
print(len(data[(data.loan == 'unknown') & (data.housing == 'unknown')]))

946
946


А переменные loan и housing являются бинарными категориальными - пропущенных значений в каждой из них и в них суммарно - 946, что составляет 1/40 от размера датасета - поэтому, просто удалим 984 наблюдения.

In [23]:
data = data[(data.loan != 'unknown') & (data.housing != 'unknown')]
data

,age,job,marital,education,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,cellular,nov,fri,383,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,cellular,nov,fri,442,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [24]:
2943/41188

0.07145285034476061

Теперь имеем:

In [25]:
for feat in categorial:
    if feat != 'default':
        print(f"Фича {feat} имеет {len(data[data[feat] == 'unknown'])} полей unknown")

Фича job имеет 0 полей unknown
Фича marital имеет 0 полей unknown
Фича education имеет 0 полей unknown
Фича housing имеет 0 полей unknown
Фича loan имеет 0 полей unknown
Фича contact имеет 0 полей unknown
Фича month имеет 0 полей unknown
Фича day_of_week имеет 0 полей unknown


Мы потеряли 41188 - 38245 = 2943 наблюдения, что составляет семь сотых от размера исходного датасета - это приемлемый результат, однако мы обработали пропущенные значения, которые могут существенно повлиять на результаты random forest classifier (который будет использован далее).

Теперь, необходимо закодировать с помощью дамми-переменными категориальные переменные, расширив датасет новым набором фич, которые будут соответствовать закодированным категориальным переменным.

Создадим новый датафрейм, сначала в него войдут только числовые фичи.

In [26]:
df = data[['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
           'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']]
df

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0
...,...,...,...,...,...,...,...,...,...,...
41183,73,334,1,999,0,-1.1,94.767,-50.8,1.028,4963.6
41184,46,383,1,999,0,-1.1,94.767,-50.8,1.028,4963.6
41185,56,189,2,999,0,-1.1,94.767,-50.8,1.028,4963.6
41186,44,442,1,999,0,-1.1,94.767,-50.8,1.028,4963.6


In [27]:
categorial_edited = ['job', 'marital', 'education', 'housing', 'loan', 'contact', 'month', 'day_of_week']

In [28]:
for feat in categorial_edited:
    print(data[feat].unique())

['housemaid' 'services' 'admin.' 'technician' 'blue-collar' 'retired'
 'management' 'unemployed' 'self-employed' 'entrepreneur' 'student']
['married' 'single' 'divorced']
['basic.4y' 'high.school' 'basic.6y' 'basic.9y' 'professional.course'
 'university.degree' 'illiterate']
['no' 'yes']
['no' 'yes']
['telephone' 'cellular']
['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']
['mon' 'tue' 'wed' 'thu' 'fri']


In [29]:
for feat in categorial_edited:
    df = pd.concat([df, pd.get_dummies(data[feat])], axis=1)
df

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,mar,may,nov,oct,sep,fri,mon,thu,tue,wed
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,334,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,1,0,0,0,0
41184,46,383,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,1,0,0,0,0
41185,56,189,2,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,1,0,0,0,0
41186,44,442,1,999,0,-1.1,94.767,-50.8,1.028,4963.6,...,0,0,1,0,0,1,0,0,0,0


Так мы собрали в один датафрейм все числовые фичи и категориальные, закодировав категориальные. Теперь, перед факторным анализом (а именно - методом главных компонент), осуществим стандартизацию данных - приведение данных к отрезку [0, 1].

In [30]:
df = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(df), columns=df.columns)
df

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,mar,may,nov,oct,sep,fri,mon,thu,tue,wed
0,0.481481,0.053070,0.000000,1.0,0.000000,0.937500,0.698753,0.60251,0.957379,0.859735,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.493827,0.030297,0.000000,1.0,0.000000,0.937500,0.698753,0.60251,0.957379,0.859735,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.246914,0.045954,0.000000,1.0,0.000000,0.937500,0.698753,0.60251,0.957379,0.859735,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.283951,0.030704,0.000000,1.0,0.000000,0.937500,0.698753,0.60251,0.957379,0.859735,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.481481,0.062424,0.000000,1.0,0.000000,0.937500,0.698753,0.60251,0.957379,0.859735,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38240,0.691358,0.067914,0.000000,1.0,0.000000,0.479167,1.000000,0.00000,0.089322,0.000000,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
38241,0.358025,0.077877,0.000000,1.0,0.000000,0.479167,1.000000,0.00000,0.089322,0.000000,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
38242,0.481481,0.038430,0.023810,1.0,0.000000,0.479167,1.000000,0.00000,0.089322,0.000000,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
38243,0.333333,0.089874,0.000000,1.0,0.000000,0.479167,1.000000,0.00000,0.089322,0.000000,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Теперь, добавим в датафрейм отклик (наш y).

In [31]:
d = pd.DataFrame(pd.get_dummies(data.y).iloc[:, 1])
d.columns = ['y']
d.index = range(len(df))
df = pd.concat([d, df], axis=1)
df

,y,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,...,mar,may,nov,oct,sep,fri,mon,thu,tue,wed
0,0,0.481481,0.053070,0.000000,1.0,0.000000,0.937500,0.698753,0.60251,0.957379,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,0.493827,0.030297,0.000000,1.0,0.000000,0.937500,0.698753,0.60251,0.957379,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,0.246914,0.045954,0.000000,1.0,0.000000,0.937500,0.698753,0.60251,0.957379,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0,0.283951,0.030704,0.000000,1.0,0.000000,0.937500,0.698753,0.60251,0.957379,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0,0.481481,0.062424,0.000000,1.0,0.000000,0.937500,0.698753,0.60251,0.957379,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38240,1,0.691358,0.067914,0.000000,1.0,0.000000,0.479167,1.000000,0.00000,0.089322,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
38241,0,0.358025,0.077877,0.000000,1.0,0.000000,0.479167,1.000000,0.00000,0.089322,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
38242,0,0.481481,0.038430,0.023810,1.0,0.000000,0.479167,1.000000,0.00000,0.089322,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
38243,1,0.333333,0.089874,0.000000,1.0,0.000000,0.479167,1.000000,0.00000,0.089322,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38245 entries, 0 to 38244
Data columns (total 53 columns):
y                      38245 non-null uint8
age                    38245 non-null float64
duration               38245 non-null float64
campaign               38245 non-null float64
pdays                  38245 non-null float64
previous               38245 non-null float64
emp.var.rate           38245 non-null float64
cons.price.idx         38245 non-null float64
cons.conf.idx          38245 non-null float64
euribor3m              38245 non-null float64
nr.employed            38245 non-null float64
admin.                 38245 non-null float64
blue-collar            38245 non-null float64
entrepreneur           38245 non-null float64
housemaid              38245 non-null float64
management             38245 non-null float64
retired                38245 non-null float64
self-employed          38245 non-null float64
services               38245 non-null float64
student            

In [33]:
df.describe()

,y,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,...,mar,may,nov,oct,sep,fri,mon,thu,tue,wed
count,38245.000000,38245.000000,38245.000000,38245.000000,38245.000000,38245.000000,38245.000000,38245.000000,38245.000000,38245.000000,...,38245.000000,38245.000000,38245.000000,38245.000000,38245.000000,38245.000000,38245.000000,38245.000000,38245.000000,38245.000000
mean,0.111335,0.282233,0.052503,0.037301,0.964496,0.024287,0.725596,0.533637,0.429240,0.677692,...,0.012995,0.334527,0.101922,0.017231,0.013126,0.188887,0.207269,0.209465,0.195607,0.198771
std,0.314550,0.127031,0.052825,0.065892,0.184480,0.069596,0.326239,0.224617,0.193439,0.392252,...,0.113255,0.471831,0.302549,0.130133,0.113815,0.391424,0.405355,0.406932,0.396673,0.399081
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.185185,0.020740,0.000000,1.000000,0.000000,0.333333,0.340608,0.338912,0.160961,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.259259,0.036600,0.023810,1.000000,0.000000,0.937500,0.484412,0.376569,0.957379,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.370370,0.064864,0.047619,1.000000,0.000000,1.000000,0.698753,0.602510,0.980957,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Теперь все переменные закодированы и являются числовыми. Можно переходить к построению первичной модели на полных данных без пропусков.

## Первичная модель для распознавания целевой переменной (отклика) (первое решение)

In [34]:
y = df["y"]
x = df.drop(columns = ["y"])

In [35]:
x.shape

(38245, 52)

Сперва заметим, что классы несбалансированны - поэтому в качестве криетрия качества выберем balanced_accuracy.

In [36]:
df['y'].value_counts()

0    33987
1     4258
Name: y, dtype: int64

In [37]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state = 1337) 

Во избежание вычислительной сложности подбора параметров gridsearch'ем, в данной лабораторной работе вместо метода подбора параметров gridsearch'ем будет использоваться, по аналогии с лекционным материалом, метод покоординатного спуска.

In [38]:
model = RandomForestClassifier(random_state=54,
                               n_estimators=200,
                               criterion='gini',
                               max_depth=10,
                               min_impurity_decrease=0.0002,
                               bootstrap=False,
                               min_samples_leaf=6,
                               min_samples_split=6,
                               n_jobs=4)

Параметры, написанные выше, являются подобранные методом покоординатного спуска. Для упрощения вычислительной сложности, в данной аналитической работе, будем считать - что данный набор параметров является "глобально" оптимальным - то есть при построении всех следующих девяти моделей случайного леса будет использоваться данный набор параметров.

In [39]:
model.fit(x_train, y_train)
y_pred_train_simple = model.predict(x_train)
y_pred_test_simple = model.predict(x_test)
probabilities_simple = model.predict_proba(x_test)

In [40]:
def get_info(model, x_train):
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    ans = []
    ans.append(np.round(accuracy_score(y_train, y_pred_train)*100, 2))
    ans.append(np.round(accuracy_score(y_test, y_pred_test)*100, 2))
    ans.append(np.round(balanced_accuracy_score(y_train, y_pred_train)*100, 2))
    ans.append(np.round(balanced_accuracy_score(y_test, y_pred_test)*100, 2))
    probab = model.predict_proba(x_test)
    ans.append(np.round(len(probab[probab > 0.8]) / len(probab), 2))
    return ans

def print_info(ans):
    print(f"Лучшее accuracy на обучающей выборке составило {ans[0]}%.")
    print(f"Лучшее accuracy на тестовой выборке составило {ans[1]}%.")
    print(f"Значение balanced_accuracy на обучающей выборке составило {ans[2]}%.")
    print(f"Значение balanced_accuracy на тестовой выборке составило {ans[3]}%.")
    print("Доля наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8, составила",
          f"{ans[3]}.")

In [41]:
results = []
results.append(get_info(model, x_train))

In [42]:
print_info(results[0])

Лучшее accuracy на обучающей выборке составило 90.58%.
Лучшее accuracy на тестовой выборке составило 90.15%.
Значение balanced_accuracy на обучающей выборке составило 59.03%.
Значение balanced_accuracy на тестовой выборке составило 59.17%.
Доля наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8, составила 59.17.


## Генерация пропущенных значений.

In [43]:
def gen_random_indexes(part):
    n, m = x_train.shape
    i = np.random.choice(n, int(n * part))
    j = np.random.choice(m, int(n * part))
    return np.transpose(np.array([i, j]))

def replace_to_nans(df, randoms):
    df_cop = df.copy()
    for i, j in randoms:
        df_cop.iloc[i, j] = np.NaN
    return df_cop

In [44]:
x_train_nans = []
for part in [0.1, 0.25, 0.33]:
    rands = gen_random_indexes(part)
    x_train_nans.append(replace_to_nans(x_train, rands))

In [45]:
x_train_nans

[            age  duration  campaign     pdays  previous  emp.var.rate  \
 3813   0.370370  0.036397  0.023810  1.000000  0.000000      0.937500   
 26916  0.271605  0.069337  0.000000  1.000000  0.000000      0.333333   
 278    0.283951  0.040464  0.047619  1.000000  0.000000      0.937500   
 9898   0.160494  0.013420  0.000000  1.000000  0.000000      1.000000   
 30351  0.111111  0.001830  0.142857  1.000000  0.142857      0.333333   
 ...         ...       ...       ...       ...       ...           ...   
 21977  0.185185  0.007930  0.119048  1.000000  0.000000      1.000000   
 19623  0.160494  0.038837  0.000000  1.000000  0.000000      1.000000   
 9448   0.148148  0.263928  0.023810  1.000000  0.000000      1.000000   
 33628  0.246914  0.045140  0.000000  0.002002  0.142857      0.333333   
 3223   0.333333  0.171614  0.000000  1.000000  0.000000      0.937500   
 
        cons.price.idx  cons.conf.idx  euribor3m  nr.employed  ...  mar  may  \
 3813         0.698753       0

## Первый способ замены пропущенных значений - средними или медианами или модами по столбцу.

In [46]:
def repalce_floats(x):
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean').fit(x)
    return pd.DataFrame(imp_mean.transform(x), columns=x.columns)

def repalce_categorial(x):
    imp_mod = SimpleImputer(missing_values=np.nan, strategy='most_frequent').fit(x)
    return pd.DataFrame(imp_mod.transform(x), columns=x.columns)

def concat_replaced(x):
    return pd.concat([repalce_floats(x.iloc[:, :10]), repalce_categorial(x.iloc[:, 10:])], axis=1)

In [47]:
skips = [0.1, 0.25, 0.33]

In [48]:
x_train_replaced_means = []
for x in x_train_nans:
    x_train_replaced_means.append(concat_replaced(x))

In [49]:
for x, part in zip(x_train_replaced_means, skips):
    model.fit(x, y_train)
    res = get_info(model, x)
    print("")
    print(f"Для {part * 100}% потерянной информации восстановление пропущенных значений средними показало следующие результаты:")
    print("")
    print_info(res)
    results.append(res)


Для 10.0% потерянной информации восстановление пропущенных значений средними показало следующие результаты:

Лучшее accuracy на обучающей выборке составило 90.62%.
Лучшее accuracy на тестовой выборке составило 90.08%.
Значение balanced_accuracy на обучающей выборке составило 59.03%.
Значение balanced_accuracy на тестовой выборке составило 58.83%.
Доля наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8, составила 58.83.

Для 25.0% потерянной информации восстановление пропущенных значений средними показало следующие результаты:

Лучшее accuracy на обучающей выборке составило 90.56%.
Лучшее accuracy на тестовой выборке составило 90.13%.
Значение balanced_accuracy на обучающей выборке составило 58.85%.
Значение balanced_accuracy на тестовой выборке составило 59.12%.
Доля наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8, составила 59.12.

Для 33.0% потерянной информации восстановление пропущ

## Второй способ замены пропущенных значений - методом k-го ближайшего соседа

In [50]:
def check_all_neighbors(x, k):
    ans = []
    for i in range(2, k):
        ans.append(KNNImputer(n_neighbors=i, weights="uniform").fit_transform(x))
    return ans

def gen_checks(x_nans, k):
    ans = []
    for x in x_nans:
        ans.append(check_all_neighbors(x, k))
    return ans

In [52]:
kneighb = gen_checks(x_train_nans, 9)

In [53]:
ans = [[] for i in range(3)]
for i in range(3):
    for x in kneighb[i]:
        model.fit(x, y_train)
        res = get_info(model, x)
        ans[i].append(res)
ans = [max(ans[i]) for i in range(len(ans))]
for i, part in enumerate(skips):
    print("")
    print(f"Для {part * 100}% потерянной информации восстановление пропущенных значений средними показало следующие результаты:")
    print("")
    print_info(ans[i])


Для 10.0% потерянной информации восстановление пропущенных значений средними показало следующие результаты:

Лучшее accuracy на обучающей выборке составило 90.66%.
Лучшее accuracy на тестовой выборке составило 90.18%.
Значение balanced_accuracy на обучающей выборке составило 59.38%.
Значение balanced_accuracy на тестовой выборке составило 59.48%.
Доля наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8, составила 59.48.

Для 25.0% потерянной информации восстановление пропущенных значений средними показало следующие результаты:

Лучшее accuracy на обучающей выборке составило 90.67%.
Лучшее accuracy на тестовой выборке составило 90.21%.
Значение balanced_accuracy на обучающей выборке составило 59.49%.
Значение balanced_accuracy на тестовой выборке составило 59.53%.
Доля наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8, составила 59.53.

Для 33.0% потерянной информации восстановление пропущ

In [54]:
for a in ans:
    results.append(a)
results

[[90.58, 90.15, 59.03, 59.17, 0.83],
 [90.62, 90.08, 59.03, 58.83, 0.82],
 [90.56, 90.13, 58.85, 59.12, 0.82],
 [90.53, 90.05, 58.56, 58.63, 0.81],
 [90.66, 90.18, 59.38, 59.48, 0.81],
 [90.67, 90.21, 59.49, 59.53, 0.82],
 [90.66, 90.14, 59.36, 59.25, 0.8]]

## Третий способ замены пропущенных значений - заполнение случайным лесом

Для заполнения случайным лесом пропусков воспользуемся библиотекой missingpy и методом MissForest.

In [55]:
x_train_replaced_forest = []
for x in x_train_nans:
    imputer = MissForest()
    x_train_replaced_forest.append(imputer.fit_transform(x))

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5
Iteration: 6
Iteration: 7
Iteration: 8


In [58]:
ans = []
for x, part in zip(x_train_replaced_forest, skips):
    model.fit(x, y_train)
    res = get_info(model, x)
    ans.append(res)
    print("")
    print(f"Для {part * 100}% потерянной информации восстановление пропущенных значений средними показало следующие результаты:")
    print("")
    print_info(res)


Для 10.0% потерянной информации восстановление пропущенных значений средними показало следующие результаты:

Лучшее accuracy на обучающей выборке составило 90.58%.
Лучшее accuracy на тестовой выборке составило 90.15%.
Значение balanced_accuracy на обучающей выборке составило 59.12%.
Значение balanced_accuracy на тестовой выборке составило 59.2%.
Доля наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8, составила 59.2.

Для 25.0% потерянной информации восстановление пропущенных значений средними показало следующие результаты:

Лучшее accuracy на обучающей выборке составило 90.58%.
Лучшее accuracy на тестовой выборке составило 90.08%.
Значение balanced_accuracy на обучающей выборке составило 59.27%.
Значение balanced_accuracy на тестовой выборке составило 59.28%.
Доля наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8, составила 59.28.

Для 33.0% потерянной информации восстановление пропущен

In [61]:
for a in ans:
    results.append(a)
results

[[90.58, 90.15, 59.03, 59.17, 0.83],
 [90.62, 90.08, 59.03, 58.83, 0.82],
 [90.56, 90.13, 58.85, 59.12, 0.82],
 [90.53, 90.05, 58.56, 58.63, 0.81],
 [90.66, 90.18, 59.38, 59.48, 0.81],
 [90.67, 90.21, 59.49, 59.53, 0.82],
 [90.66, 90.14, 59.36, 59.25, 0.8],
 [90.58, 90.15, 59.12, 59.2, 0.82],
 [90.58, 90.08, 59.27, 59.28, 0.82],
 [90.57, 90.1, 58.92, 58.99, 0.83]]

In [62]:
k = results.copy()

## Сведем полученные результаты о качестве 10 моделей в таблицу.

In [72]:
results = np.array(results)
df_results = pd.DataFrame(results)
df_results.columns = ['accuracy_train', 'accuracy_test', 'balanced_accuracy_train', 'balanced_accuracy_test', 'prob']
df_results['missing_values'] = ['0%', '10%', '25%', '33%','10%', '25%', '33%','10%', '25%', '33%']
df_results = pd.concat([pd.DataFrame(['модель без пропусков', 'модель с заполнением средними по столбцу', 'модель с заполнением средними по столбцу',
 'модель с заполнением средними по столбцу', 'модель с заполнением методом k-го ближайшего соседа', 
 'модель с заполнением методом k-го ближайшего соседа', 'модель с заполнением методом k-го ближайшего соседа',
 'модель с заполнением случайным лесом', 'модель с заполнением случайным лесом', 'модель с заполнением случайным лесом'],
                                   columns=['method']), df_results], axis=1)
df_results.style.set_properties(**{'text-align': 'center'}).set_table_styles([ dict(selector='th', props=[('text-align', 'center')] ) ])

,method,accuracy_train,accuracy_test,balanced_accuracy_train,balanced_accuracy_test,prob,missing_values
0,модель без пропусков,90.58,90.15,59.03,59.17,0.83,0%
1,модель с заполнением средними по столбцу,90.62,90.08,59.03,58.83,0.82,10%
2,модель с заполнением средними по столбцу,90.56,90.13,58.85,59.12,0.82,25%
3,модель с заполнением средними по столбцу,90.53,90.05,58.56,58.63,0.81,33%
4,модель с заполнением методом k-го ближайшего соседа,90.66,90.18,59.38,59.48,0.81,10%
5,модель с заполнением методом k-го ближайшего соседа,90.67,90.21,59.49,59.53,0.82,25%
6,модель с заполнением методом k-го ближайшего соседа,90.66,90.14,59.36,59.25,0.8,33%
7,модель с заполнением случайным лесом,90.58,90.15,59.12,59.2,0.82,10%
8,модель с заполнением случайным лесом,90.58,90.08,59.27,59.28,0.82,25%
9,модель с заполнением случайным лесом,90.57,90.1,58.92,58.99,0.83,33%


Вывод: таким образом, в ходе данной лабораторной работы были выполнены три метода заполнения пропусков:
1. средними/модой по столбцу
2. методом k-го ближайшего соседа
3. заполнение пропусков случайным лесом

Также, результаты использования модели случайного леса на всех 10-ти наборах данных были представлены в виде таблицы.
Результаты, на самом деле удивительные - все модели, построенные на разных наборах данных с разными методами заполнения имеют примерно одинаковые результаты качества. Объясняется это тем, что количество фич в исходном наборе данных равно 52, а количество наблюдений - 38 тысяч, соответственно влияние, оказываемое пропусками в 10%, 25% и 33% всех наблюдений оказалось несущественным. Сравнивая результаты, самым эффективным оказался метод k-го ближайшего соседа - он показал даже лучшие значения качества модели, по сравнению с набором данных без пропусков (отличие ~0.1-0.5%).